<center>
<hr>
<h1>Complessità nei sistemi sociali</h1>
<h2>Laurea Magistrale in Fisica Dei Sistemi Complessi</h2>
<h2>A.A. 2016/17</h2>
<h3>Dr. Daniela Paolotti, Dr. Michele Tizzoni</h3>
<h3>Basic network analysis with NetworkX, assortativity, resilience to attacks</h3>
<hr>
</center>

In [2]:
import networkx as nx
import seaborn as sns

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Measure the connectivity and clustering of a graph

In [4]:
G=nx.Graph()

In [5]:
#Download the network file 'ca-AstroPh' from SNAP@Stanford
fh=open('./ca-AstroPh.txt','r')
for line in fh.readlines():
    s=line.strip().split()
    if s[0]!='#':
        origin=int(s[0])
        dest=int(s[1])
        G.add_edge(origin,dest)
fh.close()

In [6]:
print "The graph has", len(G), "nodes"
print "and",len(G.edges()),"edges"

The graph has 18772 nodes
and 198110 edges


In [7]:
print "Is the graph simply connected?", nx.is_connected(G)

Is the graph simply connected? False


#### Show the components of the graph

In [8]:
print "The graph has", nx.number_connected_components(G),"connected components"

The graph has 290 connected components


In [9]:
for k in nx.connected_components(G):
    print len(k)

17903
4
2
3
2
3
3
9
4
2
3
2
4
2
2
2
5
2
2
6
3
3
8
2
2
5
3
8
5
4
2
3
3
3
3
4
2
4
2
3
5
3
4
3
3
4
2
3
3
2
18
3
2
2
3
5
2
4
3
2
3
3
3
12
7
3
2
3
3
2
3
2
5
4
3
3
2
10
3
3
2
2
4
4
2
8
2
2
2
2
4
3
5
5
5
2
3
5
4
4
3
5
4
2
2
3
2
5
2
2
3
2
3
3
3
2
3
3
4
3
2
3
2
2
2
2
3
3
2
2
3
2
2
3
4
2
2
7
3
2
3
2
3
4
4
3
2
4
4
2
2
2
2
5
2
2
4
2
3
4
2
2
3
3
2
2
2
2
2
2
3
4
3
4
2
10
4
2
2
4
2
2
3
2
2
3
3
6
3
2
2
3
3
2
7
2
2
4
3
3
2
2
2
2
3
3
2
2
2
2
4
2
3
2
2
2
4
2
2
4
3
2
2
3
3
2
2
6
3
3
2
2
2
2
2
3
4
3
2
2
2
2
2
3
2
3
2
2
3
2
2
4
2
4
2
2
3
2
5
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
3
2
2
3
2
3
2
2
2
2
4
3
3
1


#### Extract the largest Connected Component as a subgraph

In [10]:
nx.connected_component_subgraphs(G)

<generator object connected_component_subgraphs at 0x10f076280>

In [11]:
graphs = list(nx.connected_component_subgraphs(G))

In [12]:
graphs

In [13]:
H=graphs[0]

In [14]:
print len(G)-len(H)

869


In [15]:
print "Check that the graph is now connected"
nx.number_connected_components(H)

Check that the graph is now connected


1

## Average clustering coefficient

As an alternative to the global clustering coefficient, the overall level of clustering in a network is measured by Watts and Strogatz as the average of the local clustering coefficients of all the vertices $n$:

$\bar{C} = \frac{1}{n}\sum_{i=1}^{n} C_i.$

It is worth noting that this metric places more weight on the low degree nodes, while the transitivity ratio places more weight on the high degree nodes. In fact, a weighted average where each local clustering score is weighted by $k_i(k_i-1)$ is identical to the global clustering coefficient.

In [16]:
print "The average clustering coefficient of G is"
nx.average_clustering(G)

The average clustering coefficient of G is


0.6305932411707976

In [ ]:
nx.triangles(G)

How many triangles there are in the whole network?

In [ ]:
print float(sum(nx.triangles(G).values()))/3

#### Compare the results with a random ER network

We generate a random Erdos-Renyi graph with same average connectivity of H

In [17]:
nnodes=18000
plink=0.00122

ER=nx.fast_gnp_random_graph(nnodes, plink)

In [18]:
print "The ER graph has", len(ER), "nodes"
print "and",len(ER.edges()),"edges"

The ER graph has 18000 nodes
and 197514 edges


In [19]:
print "The average clustering coefficient of ER is"
nx.average_clustering(ER)

The average clustering coefficient of ER is


0.001216395385141159

In [ ]:
print float(sum(nx.triangles(ER).values()))/3

In [ ]:
nx.is_connected(ER)

#### Compare the results with a random AB network

In [ ]:
AB=nx.barabasi_albert_graph(18000,10)

In [ ]:
from collections import Counter #this is a subclass of dictionary

c=Counter(AB.degree().values())

In [ ]:
x=[]
y=[]
for i in sorted(c):   
    x.append(i)
    y.append(float(c[i])/len(AB))

    
plt.plot(np.array(x),np.array(y))

plt.xlabel('$k$', fontsize=18)
plt.ylabel('$P(k)$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.yscale('log')
plt.xscale('log')
plt.axis([10,1000,0.00001,1])
plt.show()

In [ ]:
print "The AB graph has", len(AB), "nodes"
print "and",len(AB.edges()),"edges"

In [ ]:
print "The average clustering coefficient of AB is"
nx.average_clustering(AB)

In [ ]:
print "The number of triangles is ", float(sum(nx.triangles(AB).values()))/3.0

## Closeness Centrality

In connected graphs there is a natural distance metric between all pairs of nodes, defined by the length of their shortest paths. 
The '''farness''' of a node ''x'' is defined as the sum of its distances from all other nodes, and its closeness was defined by Bavelas as the reciprocal of the farness that is:

$C(x)= \frac{1}{\sum_y d(y,x)}.$

Thus, the more central a node is the lower its total distance from all other nodes. Note that taking distances ''from'' or ''to'' all other nodes is irrelevant in undirected graphs, whereas in directed graphs distances ''to'' a node are considered a more meaningful measure of centrality, as in general (e.g., in, the web) a node has little control over its incoming links.


####Be careful! Computing all the distances between pair of nodes can be intensive.

In [ ]:
close_centr=nx.closeness_centrality(G)

In [ ]:
print close_centr

## Measure the assortativity of the network

Assortativity can be measured in different ways. A simple approach is measuring the average nearest neighbor degree.

In [ ]:
from collections import defaultdict

In [ ]:
x=[]
y=[]

avg_knn=defaultdict(list)

for n in G.nodes():
    k=G.degree(n)
    
    nn=len(G.neighbors(n))
    total=0
    for j in G.neighbors(n):
        total+=G.degree(j)
    
    avg_knn[k].append(float(total)/nn)
    
    x.append(k)
    y.append(float(total)/nn)

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(x,y)

plt.xlabel('$k_i$', fontsize=18)
plt.ylabel('$k_{nn}$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.yscale('log')
plt.xscale('log')
plt.axis([0.1,1000,1,1000])
plt.show()

In [ ]:
z=[]
for k in avg_knn:
    knn=np.array(avg_knn[k])
    z.append(np.average(knn))
    #print k, np.average(knn)

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(x,y)
plt.plot(avg_knn.keys(),z, color='r')

plt.xlabel('$k_i$', fontsize=18)
plt.ylabel('$k_{nn}$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.yscale('log')
plt.xscale('log')
plt.axis([0.1,1000,1,1000])
plt.show()

In [ ]:
r=nx.degree_assortativity_coefficient(G)

In [ ]:
print r

The degree assortativity coefficient of a ER graph is much smaller.

In [ ]:
ERr=nx.degree_assortativity_coefficient(ER)

In [ ]:
ERr

NetworkX offers a number of functions to compute the same quantity.

In [ ]:
knn_avg2=nx.average_degree_connectivity(G)

In [ ]:
print knn_avg2

In [ ]:
knn_avg3=nx.k_nearest_neighbors(G)

In [ ]:
print knn_avg3

In [ ]:
r2=nx.degree_pearson_correlation_coefficient(G)
print r2 

# Random and targeted attacks and resilience

Resilience is the ability to provide and maintain an acceptable level of service in the face of faults and challenges to normal operation. Threats and challenges for services can range from simple misconfiguration over large scale natural disasters to targeted attacks.

We define a function that performs a random or targeted attack to a network according to a given strategy (random, degree based, betweenness based, etc. )

In [ ]:
def net_attack(graph, ranked_nodes):
    
    fraction_removed=[]#here we store the tuples: (%removed nodes, size of gcc)
    
    graph1=graph.copy()
    nnodes=len(ranked_nodes)
    n=0    
    
    gcc=list(nx.connected_components(graph1))[0]
    
    gcc_size=float(len(gcc))/nnodes
    
    fraction_removed.append( (float(n)/nnodes, gcc_size) )
    
    while gcc_size>0.01:
        
        #we start from the end of the list!
        graph1.remove_node(ranked_nodes.pop())

        gcc=list(nx.connected_components(graph1))[0]
        gcc_size=float(len(gcc))/nnodes
        n+=1
        fraction_removed.append( (float(n)/nnodes, gcc_size) )
    
    return fraction_removed

#### Random attack

In [ ]:
airport_nodes=G.nodes()
resilience_random=net_attack(G, airport_nodes)

#### Betweenness based attack

In [ ]:
airport_nodes_betw=[]

betw=nx.betweenness_centrality(G)
for i in sorted(betw.iteritems(), key=itemgetter(1)):
    airport_nodes_betw.append(i[0])


resilience_betw=net_attack(G, airport_nodes_betw)

Let's compare the results.

In [ ]:
x=[k[0] for k in resilience_random]
y=[k[1] for k in resilience_random]

x1=[k[0] for k in resilience_deg]
y1=[k[1] for k in resilience_deg]

x2=[k[0] for k in resilience_betw]
y2=[k[1] for k in resilience_betw]

plt.plot(x,y, label='random attack')
plt.plot(x1,y1, label='degree based')
plt.plot(x2,y2, label='betw based')

plt.xlabel('$f_{c}$', fontsize=18)
plt.ylabel('$LCC$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.legend(loc='upper right')

#### Resilience of the Erdos-Renyi random network


In [ ]:
ER=nx.fast_gnp_random_graph(2000,0.012)

In [ ]:
ER_nodes=ER.nodes()

ER_nodes_deg=[]
for i in sorted(ER.degree().iteritems(), key=itemgetter(1)):
    ER_nodes_deg.append(i[0])

ER_nodes_betw=[]
ER_betw=nx.betweenness_centrality(ER)
for i in sorted(ER_betw.iteritems(), key=itemgetter(1)):
    ER_nodes_betw.append(i[0])


In [ ]:
resilience_random=net_attack(ER, ER_nodes)
resilience_deg=net_attack(ER, ER_nodes_deg)
resilience_betw=net_attack(ER, ER_nodes_betw)

In [ ]:
x=[k[0] for k in resilience_random]
y=[k[1] for k in resilience_random]

x1=[k[0] for k in resilience_deg]
y1=[k[1] for k in resilience_deg]

x2=[k[0] for k in resilience_betw]
y2=[k[1] for k in resilience_betw]

plt.plot(x,y, label='random attack')
plt.plot(x1,y1, label='degree based')
plt.plot(x2,y2, label='betw based')

plt.xlabel('$f_{c}$', fontsize=18)
plt.ylabel('$LCC$', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.legend(loc='upper right')

#### Resilience of the Barabasi-Albert random network
